# Introduction

In this notebook, I will play with FuzzyMarketState python class, testing all its features.

In [1]:
# import requires packages
# Append relative path to FuzzyMarketState.py
import sys
sys.path.append('..')

from FuzzyMarketState import FuzzyMarketState

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly.tools import FigureFactory as FF
import plotly.tools as tls
plotly.offline.init_notebook_mode(connected=True)

import logging
logging.basicConfig(level=logging.DEBUG, stream=sys.stdout)

import random

print('Packages loaded!!')

Packages loaded!!


In [2]:
# Create object
fms = FuzzyMarketState(logging.WARN)

In [3]:
# loads dataframe
df = fms.loadCSV('../csv_data/EURUSD_H4.csv', sep=';')

## Zigzag and Bollinger indicators

In [4]:
# builds zigzag indicator
df = fms.buildZigzag(df, minbars=12, bb_period=20, bb_dev=2.0, bb_sma=[100], nan_value=0.0)


In [5]:
# loads dataframe with zigzag into fms and plot the zigzag trace
fms.loadDataframe(df)
zz_traces = fms.plotZigzag('black')
py.iplot(zz_traces, filename='simple_ohlc')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


In [6]:
# plots Bollinger indicators associated with zigzag
fig = plotly.tools.make_subplots(rows=3, cols=1, subplot_titles=('Price & Bollinger', 'Bollinger derivatives'), shared_xaxes=True, vertical_spacing=0.1)
bb_traces = fms.plotBollinger(['black', 'blue', 'red'])
fig.append_trace(bb_traces[0], 1, 1)
fig.append_trace(bb_traces[1], 1, 1)
fig.append_trace(bb_traces[2], 1, 1)
fig.append_trace(bb_traces[3], 1, 1)
fig.append_trace(bb_traces[4], 2, 1)
fig.append_trace(bb_traces[5], 3, 1)
fig['layout'].update(height=600, title='Bollinger indicators')
py.iplot(fig, filename='simple_ohlc')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


### Oscillators: MACD and RSI

In [9]:
result = fms.buildOscillators(fms.getDataframe(), macd_applied='CLOSE', macd_fast=12, macd_slow=26, macd_sig=9, rsi_applied='CLOSE', rsi_period=14)

In [8]:
fig = plotly.tools.make_subplots(rows=3, cols=1, subplot_titles=('Price', 'MACD', 'RSI'), shared_xaxes=True, vertical_spacing=0.1)
osc_traces = fms.plotOscillators(color=['blue','red','green'])
fig.append_trace(osc_traces[0], 1, 1)
fig.append_trace(osc_traces[1], 2, 1)
fig.append_trace(osc_traces[2], 2, 1)
fig.append_trace(osc_traces[4], 3, 1)
fig['layout'].update(height=600, title='Oscillators')
py.iplot(fig, filename='simple_ohlc')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]



AttributeError: 'DataFrame' object has no attribute 'MACD_main'

### Moving Averages Fast-Medium-Slow

In [10]:
# build moving averages
result = fms.build3MovingAverages(fms.getDataframe(), 
                                  ma_fast_applied='CLOSE', ma_fast_period=50, ma_fast_type='SMA', 
                                  ma_mid_applied='CLOSE', ma_mid_period=100, ma_mid_type='SMA', 
                                  ma_slow_applied='CLOSE', ma_slow_period=200, ma_slow_type='SMA')

In [ ]:
# plot moving averages
ma_traces, ma_shapes = fms.plotMovingAverages(color=['blue', 'red', 'green'])
fig = go.Figure(data=[ma_traces[0],ma_traces[1],ma_traces[2],ma_traces[3]])
fig['layout'].update(height=600, title='Trends')
fig['layout'].update(shapes=ma_shapes)
#-------------
def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.High.min() - 10, in_view.High.max() + 10]
fig.layout.on_change(zoom, 'xaxis.range')
py.iplot(fig, filename='simple_ohlc')

### Fibonacci levels

In [11]:
# build common fibo levels (both retracements and extensions)
fms.buildCommonFiboLevels(fms.getDataframe(), nan_value=0.0)

In [ ]:
# plot fibo levels for a given sample
length =  fms.getDataframe().shape[0]
x = random.randint(int(length/2), length)
fibo_traces, fibo_annotations = fms.plotFiboLevels(at=x, color='black')
fig = go.Figure(data=[fibo_traces])
fig['layout'].update(height=600, title='FiboLevels at {}'.format(x))
fig['layout'].update(annotations=fibo_annotations)
def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.High.min() - 10, in_view.High.max() + 10]
fig.layout.on_change(zoom, 'xaxis.range')
py.iplot(fig, filename='simple_ohlc')

### Supports & Resistances

In [12]:
# build supports and resistances levels from last trend zigzags
_supports = fms.buildSupports(fms.getDataframe(), nan_value=0.0)
s1,s2 = _supports['support_1'],  _supports['support_2']
s1 = s1[s1 != 0.0]
s2 = s2[s2 != 0.0]
x1 = s1.index.values[-1]
x2 = s2.index.values[-2]
x1,x2


(6108, 6107)

In [ ]:
# plot supports levels for samples x1 and x2
trace_ohlc,s1_shape=fms.plotHorizontalLine(x1+1, s1.iloc[-1], color='brown', width=2, dash='dashdot')
_,s2_shape=fms.plotHorizontalLine(x2+1, s2.iloc[-1], color='violet', width=2, dash='dashdot')
fig = go.Figure(data=[trace_ohlc])
fig['layout'].update(height=600, title='Supports at {},{}'.format(x1+1,x2+1))
fig['layout'].update(shapes=[s1_shape, s2_shape])
#-------------
def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.High.min() - 10, in_view.High.max() + 10]
fig.layout.on_change(zoom, 'xaxis.range')
py.iplot(fig, filename='simple_ohlc')

In [13]:
# build supports and resistances levels from last trend zigzags
_resist = fms.buildResistances(fms.getDataframe(), nan_value=0.0)
r1,r2 = _resist['resistance_1'],  _resist['resistance_2']
r1 = r1[r1 != 0.0]
r2 = r2[r2 != 0.0]
x1 = r1.index.values[-1]
x2 = r2.index.values[-2]
x1,x2

(6625, 6624)

In [ ]:
# plot supports levels for samples x1 and x2
trace_ohlc,r1_shape=fms.plotHorizontalLine(x1+1, r1.iloc[-1], color='brown', width=2, dash='dashdot')
_,r2_shape=fms.plotHorizontalLine(x2+1, r2.iloc[-1], color='violet', width=2, dash='dashdot')
fig = go.Figure(data=[trace_ohlc])
fig['layout'].update(height=600, title='Resistances at {},{}'.format(x1+1,x2+1))
fig['layout'].update(shapes=[r1_shape, r2_shape])
#-------------
def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.High.min() - 10, in_view.High.max() + 10]
fig.layout.on_change(zoom, 'xaxis.range')
py.iplot(fig, filename='simple_ohlc')

### Channels

In [14]:
# build channel lines
_channel = fms.buildChannel(fms.getDataframe(), nan_value='')
_upperline = _channel['channel_upper_limit']
_bottomline = _channel['channel_lower_limit']
_upperline = _upperline[_upperline != '']
_bottomline = _bottomline[_bottomline != '']
_ux = _upperline.index.values[-1]
_bx = _bottomline.index.values[-1]
_ulast = _upperline.iloc[-1]
_blast = _bottomline.iloc[-1]
_ux,_bx,_ulast,_blast


(6625, 6625, 'P3,P1', 'P4,P2')

In [ ]:
# plot channel lines for sample _ux or _bx
x = _ux+1
trace_ohlc, ch_shapes = fms.plotChannel(x, extended=100, color='black', width=1, dash='dashdot')
fig = go.Figure(data=[trace_ohlc])
fig['layout'].update(height=600, title='Channel at {}'.format(x))
fig['layout'].update(shapes=ch_shapes)
#-------------
def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.High.min() - 10, in_view.High.max() + 10]
fig.layout.on_change(zoom, 'xaxis.range')
py.iplot(fig, filename='simple_ohlc')

### Trends

In [15]:
# build trends with different filters
#filters = ['SMA_TRENDS','ZIGZAG_TRENDS','FIBO_TRENDS']
filters = ['SMA_TRENDS']
_trends = fms.buildTrends(fms.getDataframe(), filters = filters, nan_value=0.0)

In [ ]:
# plot trends
trace_ohlc, trend_shapes = fms.plotTrends(nan_value=0.0)
fig = go.Figure(data=[trace_ohlc])
fig['layout'].update(height=600, title='Trends')
fig['layout'].update(shapes=trend_shapes)
#-------------
def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.High.min() - 10, in_view.High.max() + 10]
fig.layout.on_change(zoom, 'xaxis.range')
py.iplot(fig, filename='simple_ohlc')

### Divergences

In [16]:
# build divergences
_bullish_div, _bearish_div = fms.buildDivergences(fms.getDataframe(), nan_value=0.0)

In [ ]:
# plot divergences
div_traces, div_shapes = fms.plotDivergences()